In [26]:
import numpy as np
import scipy.io
import math

# def shuffle(y1, y2):
#     p = np.random.permutation(y1.shape[0])
#     return y1[p], y2[p]

DATA_PINES= scipy.io.loadmat(r"C:\Users\Ayush\Downloads\Indian_pines_corrected.mat")
Ground_table_pines = scipy.io.loadmat(r"C:\Users\Ayush\Downloads\Indian_pines_gt.mat")
learning_rate = 0.5
lmd= 0.0001
DATA = DATA_PINES["indian_pines_corrected"]
Ground_truth = Ground_table_pines["indian_pines_gt"]


##########################reshaping  data Of size (145*145*200) to (21025*2)#################################################
DATA = DATA.reshape(21025, 200)
DATA= DATA.astype(np.float64)
Ground_truth = Ground_truth.reshape(21025)

##############Removing all elements of class 0, i.e, with 0 value in the ground truth table################################## 
indices = np.nonzero(Ground_truth)
Ground_truth = Ground_truth[indices]
DATA = DATA[indices]
Ground_truth = np.subtract(Ground_truth, 1)

###########################################Converting into one hot encoded###################################################
OneHotGt = np.zeros((Ground_truth.shape[0],16))
for i in range(0, Ground_truth.shape[0]):
    OneHotGt[i][Ground_truth[i]] = 1

#################################################scaling of feature###########################################################
for f in range(0, DATA.shape[1]):
    mean = np.mean(DATA[:,f])
    minimum = np.min(DATA[:,f])
    maximum = np.max(DATA[:,f])
    DATA[:,f] = (DATA[:,f]-mean)/(maximum- minimum)


#########################################Splitting in train and test DATA_PINES################################################
train_ground = Ground_truth[0:int(Ground_truth.shape[0]*0.5)]
test_ground = Ground_truth[int(Ground_truth.shape[0]*0.5):Ground_truth.shape[0]]
train_DATA_PINES= DATA[0:int(DATA.shape[0]*0.5)]
test_DATA_PINES= DATA[int(DATA.shape[0]*0.5):DATA.shape[0]]
train_Ground_table_pines = OneHotGt[0:int(Ground_truth.shape[0]*0.5)]
test_get = OneHotGt[int(Ground_truth.shape[0]*0.5):Ground_truth.shape[0]]

################################################randomly initialising weights##################################################
weights = np.random.randn(16,200)

################################################# doing gradient descent #######################################################
for epoch in range(0, 60000):
    print("epoch", epoch+1)
    M = np.matmul(train_DATA_PINES, np.transpose(weights))
    exp_M = np.exp(M)
    Loss = 0
########################### calculating loss by summing for each class for each training example ##############################
    for i in range(0, exp_M.shape[0]):
        sum = (np.sum(exp_M[i,:]))
        j = train_ground[i]
        Loss= Loss+ np.log((exp_M[i][j]/sum))

    Loss = (-Loss/train_DATA_PINES.shape[0]) + lmd*np.sum(np.sum(np.matmul(weights, np.transpose(weights))))
    print(" loss Training", Loss)
    probability = np.zeros((train_DATA_PINES.shape[0],16))
    for i in range(0, train_DATA_PINES.shape[0]):
        sum = (np.sum(exp_M[i,:]))
        for j in range(0, 16):
            probability[i][j] = exp_M[i][j]/sum

###############################################Testing accuracy###############################################################
    M_test = np.matmul(test_DATA_PINES, np.transpose(weights))
    exponential_of_M_test = np.exp(M_test)
    probability_test = np.zeros((test_DATA_PINES.shape[0],16))
    for i in range(0, test_DATA_PINES.shape[0]):
        sum = (np.sum(exponential_of_M_test[i,:]))
        for j in range(0, 16):
            probability_test[i][j] = exponential_of_M_test[i][j]/sum
    predict = np.argmax(probability_test, axis =1)
    accuracy = float(np.sum(predict==test_ground)/ test_DATA_PINES.shape[0])
    count=0
    for a in range(0, predict.shape[0]):
        if(predict[a]==test_ground[a]):
            count = count+1
    print("Accuracy", float(count)*100/test_DATA_PINES.shape[0])


    #weight update
    grad = np.zeros((16,200))
    grad = np.matmul(np.transpose(train_DATA_PINES), train_Ground_table_pines-probability)
    weights = weights- (learning_rate*np.transpose(grad)*(-1)/train_DATA_PINES.shape[0]) + lmd*weights
    
    print("\n")

epoch 1
 loss Training 6.0475715753868595
Accuracy 6.2439024390243905


epoch 2
 loss Training 5.414247521905353
Accuracy 5.8341463414634145


epoch 3
 loss Training 4.878615767576241
Accuracy 5.5024390243902435


epoch 4
 loss Training 4.44556526013357
Accuracy 5.170731707317073


epoch 5
 loss Training 4.098833194029449
Accuracy 5.11219512195122


epoch 6
 loss Training 3.8136946420286955
Accuracy 6.126829268292683


epoch 7
 loss Training 3.5727774340866136
Accuracy 8.175609756097561


epoch 8
 loss Training 3.3664486121815553
Accuracy 11.121951219512194


epoch 9
 loss Training 3.1887811404768445
Accuracy 13.365853658536585


epoch 10
 loss Training 3.0355559976884585
Accuracy 14.263414634146342


epoch 11
 loss Training 2.9036325899348907
Accuracy 14.009756097560976


epoch 12
 loss Training 2.790695203372614
Accuracy 13.502439024390243


epoch 13
 loss Training 2.6950641908382837
Accuracy 12.64390243902439


epoch 14
 loss Training 2.6154398286635745
Accuracy 13.580487804878048

